# Full Code

In [2]:
model_path = "/kaggle/input/exercaisor-full-code/exercaisor_model.weights.h5"
dataset_path = "/kaggle/input/exercaisor-dataset-1/dataset-final.jsonl"

In [4]:
import os
import json
import keras
import keras_nlp
import random

from tensorflow.keras import mixed_precision

mixed_precision.set_global_policy('mixed_float16')

# Set up environment variables (use these as appropriate for your system)
os.environ["KAGGLE_USERNAME"] = "farreltobias"  # replace with your actual username
os.environ["KAGGLE_KEY"] = "dbb375eb54a1babafa80f1466270a1ab"  # replace with your actual key
os.environ["KERAS_BACKEND"] = "jax"
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.00"

# Function to load dataset from JSONL
def load_data(jsonl_file):
    data = []
    with open(jsonl_file) as file:
        for line in file:
            features = json.loads(line)
            # Format the entire example as a single string
            template = "Instruction:\n{instruction}\n\nResponse:\n{response}"
            data.append(template.format(**features))
    return data

# Load the dataset
data = load_data(dataset_path)
random.shuffle(data)

# Load the pre-trained model
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma2_2b_en")
gemma_lm.summary()

# Enable LoRA for the model and set the LoRA rank to 8
gemma_lm.backbone.enable_lora(rank=8)
gemma_lm.summary()

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,614,341,888 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

...

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,620,199,168 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,620,199,168 (9.76 GB)

 Trainable params: 5,857,280 (22.34 MB)

 Non-trainable params: 2,614,341,888 (9.74 GB)

In [5]:
# Limit the input sequence length to control memory usage
gemma_lm.preprocessor.sequence_length = 256

# Use AdamW optimizer
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
    clipnorm=1.0
)
# Exclude layernorm and bias terms from decay
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

# Compile the model with Sparse Categorical Crossentropy loss and accuracy metric
gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Fine-tune the model for 1 epoch 
def fine_tune_model(data, epochs=1):
    gemma_lm.fit(data, epochs=epochs, batch_size=1)
    print("Model fine-tuning complete.")

# Save the trained model
def save_model(path="exercaisor_model.weights.h5"):
    gemma_lm.save_weights(path)
    print(f"Model saved at {path}.")
    
# Load the model
def load_model(path=model_path):
    gemma_lm.load_weights(path)
    print(f"Model loaded from {path}.")

# Run fine-tuning and save the model
if not os.path.exists(model_path):
    fine_tune_model(data)
    save_model()

# Load the model if already saved
load_model()

Model loaded from /kaggle/input/exercaisor-full-code/exercaisor_model.weights.h5.


/opt/conda/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'loss_scale_optimizer', because it has 4 variables whereas the saved optimizer has 214 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/opt/conda/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adamw', because it has 2 variables whereas the saved optimizer has 210 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
def save_model(path="exercaisor_model.weights.h5"):
    gemma_lm.save_weights(path)
    print(f"Model saved at {path}.")

save_model()

In [ ]:
# Load the model
def load_model(path=model_path):
    gemma_lm.load_weights(path)
    print(f"Model loaded from {path}.")

In [18]:
# Function to generate new variations using the fine-tuned model
template = "Instruction:\n{instruction}\n\nResponse:\n{response}"
def generate_variation(prompt):
    variations = []
    prompt = template.format(
          instruction=prompt,
          response="",
        )
    for i in range(3):
        sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2+i)
        gemma_lm.compile(sampler=sampler)
        variation = gemma_lm.generate(prompt, max_length=256)
        print(variation)
        response_part = variation.split("Response:")[1].strip()
        variations.append(response_part)
    print("Variation: ")
    for i, var in enumerate(variations):
        print(f"{i+1}. {var}")

In [ ]:
# Example prompt for generating a new variation
prompt = input("Input your problem: ")
generate_variation(prompt)

Input your problem:  Sarah has 25 marbles, and her friend gave her 37 more. Then, she bought 15 marbles from the store. Later, she lost 5 marbles while playing and found 3 marbles in her drawer. How many marbles does Sarah have now?


Instruction:
Sarah has 25 marbles, and her friend gave her 37 more. Then, she bought 15 marbles from the store. Later, she lost 5 marbles while playing and found 3 marbles in her drawer. How many marbles does Sarah have now?

Response:
Emily has 32 marbles, and her friend gave her 31 more. Then, she bought 17 marbles from the store. Later, she lost 6 marbles while playing and found 2 marbles in her locker. How many marbles does Emily have now?
Instruction:
Sarah has 25 marbles, and her friend gave her 37 more. Then, she bought 15 marbles from the store. Later, she lost 5 marbles while playing and found 3 marbles in her drawer. How many marbles does Sarah have now?

Response:
Emily has 27 marbles, and her friend gave her 18 more. Then, she bought 12 marbles from the store. Later, she lost 4 marbles while playing and found 4 marbles in her backpack. How many marbles does Emily have now?
Instruction:
Sarah has 25 marbles, and her friend gave her 37 more. Then, she bought 15 marbles from t